In [1]:
path = 'X:\CODER\ParserService\requirements.txt'
import asyncio
import httpx
from uuid import UUID, uuid4
from pydantic import BaseModel, Field

from sensory_data_client import DataClient, get_settings, DataClientConfig, PostgresConfig, MinioConfig, create_data_client
from sensory_data_client.models.document import DocumentCreate, DocumentMetadata

In [2]:
#config = DataClientConfig(PostgresConfig(host=5422))

In [ ]:
client.upload_file('X:\CODER\ParserService\X5Group x SensoryLAB 07.02.2024.pptx.pdf', )

In [4]:
import asyncio
from pathlib import Path  # Используем pathlib для удобной и кросс-платформенной работы с путями
import uuid
from typing import Optional, Coroutine, Any

from pydantic import BaseModel, Field

client =create_data_client(DataClientConfig(postgres = PostgresConfig(user = "postgres",
                                                           password = "postgres",
                                                           host = "localhost",
                                                           port=5422), 
                                            minio = MinioConfig(endpoint="localhost:9008",  bucket = "corporate")))


endpoint='localhost:9008' accesskey='minioadmin' secretkey='minioadmin' bucket='corporate' secure=False


In [6]:

# Шаг 1: Определяем путь к файлу
# Использование pathlib.Path предпочтительнее, чем простые строки.
# Он сам разберется со слэшами в Windows/Linux.
file_path_str = r'X:\CODER\ParserService\tests\X5Group x SensoryLAB 07.02.2024.pptx.pdf'
file_path = Path(file_path_str)

# Проверка, что файл существует, перед тем как его читать
if not file_path.is_file():
    print(f"Ошибка: Файл не найден по пути {file_path}")

print(f"Подготовка к загрузке файла: {file_path}")

# Шаг 2: Читаем файл в байты
# .read_bytes() - простой способ прочитать весь файл в память
try:
    file_content: bytes = file_path.read_bytes()
except Exception as e:
    print(f"Не удалось прочитать файл: {e}")
    

DOC_ID = uuid4()
# Шаг 3: Создаем объект метаданных DocumentCreate
# Извлекаем информацию из пути и добавляем остальную
document_meta = DocumentCreate(
    user_document_id=str(uuid.uuid4()), # Генерируем уникальный ID для этого документа
    name=file_path.name,                # 'X5Group x SensoryLAB 07.02.2024.pptx.pdf'
    owner="user-alpha-007",             # ID пользователя, который загружает файл
    access_group="x5-auditors",         # Опциональная группа доступа
    extension=file_path.suffix.lstrip('.'), # '.pdf' -> 'pdf'
    metadata=None
)


Подготовка к загрузке файла: X:\CODER\ParserService\tests\X5Group x SensoryLAB 07.02.2024.pptx.pdf


In [8]:

# Шаг 4: Вызываем асинхронный метод с `await`
try:
    # Ваш метод `upload_file` скорее всего захочет получить чистое имя файла,
    # а не весь путь. file_path.name идеально для этого подходит.
    result_document = await client.upload_file(
        file_name=file_path.name,
        content=file_content,
        meta=document_meta
    )
    
    print("\n✅ Успешно загружен документ!")
    print(f"   ID в базе: {result_document.id}")
    print(f"   Путь в хранилище: {result_document}")

except Exception as e:
    print(f"❌ Произошла ошибка во время загрузки: {e}")


❌ Произошла ошибка во время загрузки: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.NotNullViolationError'>: null value in column "content_hash" of relation "documents" violates not-null constraint
DETAIL:  Failing row contains (08389589-1384-4e82-a43a-826fbee58105, a555b03f-7615-4747-a23d-1db888571030, X5Group x SensoryLAB 07.02.2024.pptx.pdf, user-alpha-007, x5-auditors, pdf, null, null, null, null, 2025-08-06 14:50:38.622316+00, 2025-08-06 14:50:38.622316+00, d4daf0397e06cc9fab85daa48944e98f7ba1ef7c9a94e0ecfd322988cb3c277c).
[SQL: INSERT INTO documents (id, user_document_id, name, owner, access_group, extension, content_hash, object_path, md_object_path, stored_file_hash) VALUES ($1::UUID, $2::VARCHAR, $3::VARCHAR, $4::VARCHAR, $5::VARCHAR, $6::VARCHAR, $7::VARCHAR, $8::VARCHAR, $9::VARCHAR, $10::VARCHAR) RETURNING documents.created, documents.edited]
[parameters: (UUID('08389589-1384-4e82-a43a-826fbee58105'), 'a555b03f-7615-4747-a23d-1db88857103

In [19]:
raw_bytes = await client.get_file(UUID('cd85f28e-0eff-4767-90fd-75b5c16992f9'))

DocumentNotFoundError: 

In [18]:
from src.parser_client import DocParserClient
DOC_ID = 'cd85f28e-0eff-4767-90fd-75b5c16992f9'
PARSER_URL = "http://localhost:8000" # URL нашего doc-parser сервиса
FILE_NAME = "X5Group x SensoryLAB 07.02.2024.pptx.pdf"

# 2. Запустить парсинг и дождаться результата
parser_client = DocParserClient(base_url=PARSER_URL)
try:
    result = await parser_client.parse_and_wait(doc_id=DOC_ID, file_name=FILE_NAME)
    print("\n--- Final Result ---")
    print(result) # -> {'lines_count': 150, 'images_count': 3}
except (asyncio.TimeoutError) as e:
    print(f"\n--- Error --- \n{e}")

Client: Starting parsing for doc_id=cd85f28e-0eff-4767-90fd-75b5c16992f9, file_name='X5Group x SensoryLAB 07.02.2024.pptx.pdf'...
Client: Polling status for cd85f28e-0eff-4767-90fd-75b5c16992f9... [  0%] FAILURE | Stage: DOWNLOADING


DocParserError: Parsing failed for doc cd85f28e-0eff-4767-90fd-75b5c16992f9: ConnectionRefusedError: [Errno 111] Connection refused